In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from sklearn import tree
from copy import deepcopy
from sklearn.ensemble import BaggingRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import cross_validation
from sklearn.dummy import DummyRegressor
from scipy.optimize import minimize_scalar
%matplotlib inline

В этом ноутбуке никакие фичи не выкидываются. В моей первой посылке на ~75 выкидывалась куча данных, применялся  BaggingRegressor(n_estimators=1, ...), но эта посылка без выкидывания фичей совсем и с ExtraTreesRegressor(n_estimators=1, ...) оказалась лучше

## Данные

In [2]:
# data = pd.read_csv('blogData_train.csv', header=None)
data = pd.read_csv('./train.csv', sep=';', index_col='ID')
data.shape

(40000, 33)

In [3]:
# Посмотрим на данные
data.head()

,DEMAND,LOCATION_ID,MODEL_BRAND,MODEL_COST_AVG,MODEL_COUNT_SALES_DAYS,MODEL_DIAG,MODEL_DIAG_TYPE,MODEL_HEIGHT,MODEL_ID,MODEL_MEAN_SALES,...,STORE_FORMAT,STORE_OPEN_DTTM,STORE_PRICE_TYPE,STORE_REGION,STORE_RES_ABC,STORE_RES_SPECTR,STORE_TYPE_A,STORE_TYPE_B,STORE_TYPE_C,STORE_TYPE_SPECTR
ID,,,,,,,,,,,,,,,,,,,,,
0,0.10536,1835543262,ark,5179.7500,NaN,5.5,big,720,1682,NaN,...,новый формат,19MAY15:00:00:00,розничная цена (северо-запад-санкт-петербург),северо-запад,B,0.67,0.75,0.25,0,3.06
1,0.00010,1972241965,samsung,22015.3300,NaN,5.2,big,1920,34706,NaN,...,новый формат,20JAN16:00:00:00,розничная цена (северо-запад-днс1),северо-запад,C,0.48,0.00,1.00,0,2.51
2,0.00010,2027986985,samsung,15592.5900,NaN,5.0,big,1280,2753,NaN,...,новый формат,27APR16:00:00:00,розничная цена (дальний восток-якутск),дальний восток,A,0.64,1.00,0.00,0,2.72
3,0.00010,28432,alcatel,540.9533,29,1.8,small,128,1593,7.533,...,стандарт,21OCT09:00:00:00,розничная цена (северо-запад-боровичи),северо-запад,B,0.65,0.50,0.25,0,2.04
4,0.00010,28547,мтс,444.7300,30,4.0,medium,480,80644,16.830,...,стандарт,20MAY09:00:00:00,розничная цена (северо-запад-санкт-петербург),северо-запад,A,0.83,0.75,0.25,0,2.37


## Категориальные признаки

In [4]:
Categorical_columns = data.columns[([isinstance(z, str) for z in data.ix[0]])]
Categorical_columns

Index(['MODEL_BRAND', 'MODEL_DIAG_TYPE', 'MODEL_NAME', 'MODEL_OS',
       'MODEL_RES_TYPE', 'MODEL_TYPE', 'STORE_BRAND_ABC',
       'STORE_BUILDING_TYPE', 'STORE_CITY', 'STORE_CLUSTER', 'STORE_DIAG_ABC',
       'STORE_FORMAT', 'STORE_OPEN_DTTM', 'STORE_PRICE_TYPE', 'STORE_REGION',
       'STORE_RES_ABC'],
      dtype='object')

## Функции

In [5]:
def smape_loss_func(x, y):
    SymAPE = [ 2*np.abs(x-y)/(np.abs(x)+np.abs(y))] 
    return np.mean(SymAPE)

def get_numerated_data(df, categorical_columns):
    simple_df = deepcopy(df)
    for column in categorical_columns:
        simple_df[column] = pd.Categorical.from_array(simple_df[column]).labels
    
    return simple_df

def get_binarized_data(df, categorical_columns):
    binary_df = deepcopy(df)
    for column in categorical_columns:
        binary_df = pd.concat([binary_df, pd.get_dummies(binary_df[column], 
                                prefix=column, prefix_sep=': ')], axis=1)
        del binary_df[column]
    
    return binary_df

def split_to_numpy(df, y_column_name):
    #print df
    #print y_column_name
    data_x = df.drop(y_column_name, axis=1).values.astype(np.float32)
    data_y = df[y_column_name].values.astype(np.float32)
    column_names = df.drop(y_column_name, axis=1).columns
    
    return data_x, data_y, column_names

def split_train_test(df, test_size=0.5, random_state=42, y_column_name='y'):
    X = np.asarray(df.drop(y_column_name, axis=1))
    y = np.asarray(df[y_column_name]).ravel()
    trainX, testX, trainY, testY = cross_validation.train_test_split(X, y, 
                                                                     test_size=test_size, random_state=42)
    return trainX, testX, trainY, testY

In [6]:
def get_results_erte(n_est, verbose=True,acc=smape_loss_func):
    results = dict()
    for i in n_est:
        reg = ExtraTreesRegressor(n_estimators=i, n_jobs=-1, random_state=12, verbose=0)
        reg.fit(trainX, trainY)
        predict = reg.predict(testX)
        results[i] = (acc(trainY,reg.predict(trainX)), acc(predict, testY))
        if verbose:
            print("Count of estimators: " + str(i) + ", accuracy: " + str(results[i][1]))
    return results

### Обучение и Контроль

In [7]:
Categorical_columns

Index(['MODEL_BRAND', 'MODEL_DIAG_TYPE', 'MODEL_NAME', 'MODEL_OS',
       'MODEL_RES_TYPE', 'MODEL_TYPE', 'STORE_BRAND_ABC',
       'STORE_BUILDING_TYPE', 'STORE_CITY', 'STORE_CLUSTER', 'STORE_DIAG_ABC',
       'STORE_FORMAT', 'STORE_OPEN_DTTM', 'STORE_PRICE_TYPE', 'STORE_REGION',
       'STORE_RES_ABC'],
      dtype='object')

In [8]:
data.shape

(40000, 33)

Заполняем nan:

In [9]:
data['STORE_ASSORTMENT'] = data['STORE_ASSORTMENT'].fillna(255.)
data.fillna(0, inplace = True)

In [11]:
bin_data = get_binarized_data(data, Categorical_columns)
print(bin_data.shape)

(40000, 867)


In [12]:
# Делим выборку на обучение и контроль
trainX, testX, trainY, testY = split_train_test(df = bin_data, test_size=0.1,
                                                y_column_name='DEMAND')

Перепробовала все алгоритмы из дефолтного ноутбука (и не только их), лучший получился такой:

In [13]:
n_est = range(1, 10, 1)
%time results = get_results_erte(n_est)
show_plot(n_est, results)

Count of estimators: 1, accuracy: 0.717069051052
Count of estimators: 2, accuracy: 0.76738409083
Count of estimators: 3, accuracy: 0.804737693172


KeyboardInterrupt: 

## Отправка на Kaggle

In [950]:
test = pd.read_csv('./test.csv', sep=';',decimal= '.', index_col='ID')

In [951]:
test.fillna(0, inplace = True)

In [952]:
test_data = get_binarized_data(test, Categorical_columns)
print(test_data.shape)

(11296, 782)


In [953]:
# Все отсутствующие в тестовой выборке столбцы полагаем 0
add_columns = bin_data.columns[1:].difference(test_data.columns)
print(add_columns)
test_data = pd.merge(test_data, bin_data.ix[test_data.index][add_columns], 
                     how='inner', left_index=True, right_index=True)
test_data[add_columns]=0

# Оставляем только столбцы обучающей выборки (целевая переменная в обучающей выборке отсутствует)
test_data = test_data[bin_data.columns[1:]]

test_data.fillna(-10, inplace=True)

Index(['MODEL_BRAND: archos', 'MODEL_BRAND: bright&quick',
       'MODEL_BRAND: haier', 'MODEL_BRAND: keneksi', 'MODEL_BRAND: micromax',
       'MODEL_BRAND: prestigio', 'MODEL_NAME: acer liquid m220 dual sim',
       'MODEL_NAME: alcatel ot4015d', 'MODEL_NAME: alcatel ot5038d',
       'MODEL_NAME: alcatel ot6040d idol x dual sim',
       ...
       'MODEL_NAME: samsung s6790 galaxy fame lite',
       'MODEL_NAME: samsung s7390 galaxy trend mts',
       'MODEL_NAME: sony c1905 xperia m', 'MODEL_NAME: sony c6902 xperia z1',
       'MODEL_NAME: sony c6903 xperia z1', 'MODEL_NAME: texet tm-4172 x-star',
       'MODEL_NAME: zte grand x quad', 'MODEL_NAME: мтс 665',
       'MODEL_NAME: мтс 970 sim lock hybrid', 'MODEL_NAME: мтс 978 sim lock'],
      dtype='object', length=111)


In [954]:
test_data.shape

(11296, 866)

In [955]:
# Обучающая выборка
trainX, _, trainY, _ = split_train_test(df = bin_data, test_size=0,  
                                        y_column_name='DEMAND')

In [959]:
# Обучаем финальный алгоритм
reg = ExtraTreesRegressor(n_estimators=1, n_jobs=-1, random_state=12, verbose=0)
reg.fit(trainX, trainY)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=1, n_jobs=-1, oob_score=False, random_state=12,
          verbose=0, warm_start=False)

In [960]:
# Предсказание интенсивности
test_predict = reg.predict(np.asarray(test_data))

In [961]:
# Сохраняем в csv-файл
test['DEMAND'] = test_predict
test[['DEMAND']].to_csv('submission_gbdt.csv', sep = ',', index = True)